In [11]:
def yeardi_mesozoop(spacing,stn_b,stn_e,year):
    
    import map_fxn as mf
    import time
    import xarray as xr
    import numpy as np
    print('Spacing between stations: ' + str(spacing))
    
    bath = '/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc'
    grid = mf.import_bathy(bath)
    tmask = (grid.fmask[:,:,:,:])
    
    stn_x, stn_y = mf.make_stns(spacing)
    d_stn_x, d_stn_y = mf.filter_stn_in_domain(stn_x,stn_y,fmask)
    
    no_stns = len(d_stn_x)
    monlist = ['jan','feb','mar','apr','may','jun','jul','aug','sep',
              'oct','nov','dec']

    if year == 2016: 
        daylist = [31,29,31,30,31,30,31,31,30,31,30,31]
        noday = 366  
    else:
        daylist = [31,28,31,30,31,30,31,31,30,31,30,31]
        noday = 365  
        
    print(year)
    print(noday)
    
    # a list of all the model outputs (tracers) for this year
    tracer_list = mf.create_physlist(year)
    
    print("Number of stations:" + str(no_stns))
    bath = '/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc'
    grid = mf.import_bathy(bath)
    tmask = (grid.fmask[:,:,:,:])

    for stn in range(stn_b,stn_e):
    
        print('station is: ' ,str(stn))
        print('x is :', d_stn_x[stn])
        print('y is :', d_stn_y[stn])
        
        ts_x = d_stn_x[stn]
        ts_y = d_stn_y[stn]
        
        daily_mesozoop = np.zeros(noday)
        
        end_day = noday
        for day in range(1,end_day+1):
            if (day%20 ==0) :
                print(day)
            trc = tracer_list[day-1]
            #print(trc)
            nemo = xr.open_mfdataset(trc)
            meso = di_mesozoop(nemo,grid,ts_x,ts_y)
            q = isinstance(meso, (np.ndarray))
            if (q == True) :
                print('found an array!')
                daily_mesozoop[day-1] = meso[0]
            else:
                daily_mesozoop[day-1] = meso

        meso = xr.Dataset({'mesozooplankton':(['ptrc'], daily_mesozoop)})
        stn_name = '/ocean/ksuchy/MOAD/analysis-karyn/notebooks/CLUSTER_201905/NC_HINDCAST/' +  str(year)+ '/DI_MESOZOOP/stn_' + str(stn)  + 'di_mesozoop_sp' + str(spacing)+ '.nc'
        meso.to_netcdf(stn_name)
        
def di_mesozoop(nemo,grid,ts_x,ts_y):
        import numpy as np
    
        #retreive distances between grid cells ("distance between this grid cell and the one before it")

        delt_z = grid.variables['e3t'][0,:,ts_y:ts_y+1,ts_x:ts_x+1]
        #remove the stupid singletons
        delt_z = delt_z.values
        delt_z = np.squeeze(delt_z)
    
        zoop = nemo.mesozooplankton[0,:,ts_y:ts_y+1,ts_x:ts_x+1]
        depth = nemo.deptht
        dv = depth.values[:]
        zv = zoop.values[:]
        zv = np.squeeze(zv)

        #for this grid cell, where do we no longer have zooplankton values?
        bottom = np.where(zv ==0)
        bottom = bottom[0]
        bottom = bottom[0]

        fxnl_zoop = zv[0:bottom]
        fxnl_e3t = delt_z[0:bottom]
        fxnl_depth = dv[0:bottom]
    
        sal_gradient = np.zeros_like(fxnl_depth)

        #calculating depth-integrated mesozooplankton
        for i in range(1,bottom):
            di_zoop[i] = (fxnl_zoop[i]*fxnl_e3t[i])
        
        
        zooplankton = fxnl_depth[di_zoop]

    
        return zooplankton